https://github.com/sethih10/Interpreting-Heat-Wave-Equation-using-LLM.git

# Note - The data is generated using Finite Element Method which can be found in the data generation file. Here we are just loading the generated data.

Initially, I created a dataset which contains prompt with vtk data as text data and response. Later, I tried to finetune it but given the length of text, it was difficult to finetune and do inference. However, I have written the approach which I initially used. As submission, I have made a minute change in the reference prompt to just make a submission. 

In [1]:
import os

In [2]:
#Switching to the required directory for saving the output
os.chdir("../working")

In [3]:
!pip install transformers==4.46.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.3 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 91.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 50.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [4]:
!pip install datasets accelerate bitsandbytes peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 19.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 20.0 MB/s eta 0:00:00


In [5]:
!pip install pyvista

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 51.5 MB/s eta 0:00:0000:01


In [6]:
import transformers

#Checking the version of transformers package
print(transformers.__version__)

4.46.0


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig

In [8]:
import pandas as pd
import pyvista as pv 
import vtk
import numpy as np

In [9]:
device = "auto"
model_path = "ibm-granite/granite-3.1-2b-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)

tokenizer_config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

### Loads a quantized model with 4-bit precision (nf4) using Hugging Face and BitsAndBytes for memory-efficient inference, while timing the loading process

In [10]:
import torch
torch.cuda.empty_cache()

In [30]:
import timeit
start_time = timeit.default_timer()
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16 # if not set will throw a warning about slow speeds when training
)

model = AutoModelForCausalLM.from_pretrained(
  model_path,
  quantization_config=bnb_config,
  device_map="auto"

)

model_loadtime = timeit.default_timer() - start_time

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:

# Helper functions related to data generation

# Function to extract vtk data as pandas format
def extract_vtk_data(folder_path, case_number):
    file_path = os.path.join(folder_path,f'Case{case_number:02}.vtk')
    grid = pv.read(file_path)
    
    # Extract the grid points (coordinates)
    coordinates = grid.points  # Nx3 array of (x, y, z) coordinates
    coordinates = np.array(coordinates)
    coordinates = np.round(coordinates, 2)
    
    # Extract the scalar field (e.g., temperature values)
    temperatures = grid["Temperature"]  # Array of scalar values
    temperatures = np.array(temperatures)
    temperatures = np.round(temperatures).astype(int)

    # Create a dictionary to store temperatures at (x, y) positions
    data = {}
    for (x, y, _), temp in zip(coordinates, temperatures):
        data[(x, y)] = temp

    # Extract unique x and y values, and sort them accordingly
    unique_x = sorted(set([x for x, y in data.keys()]))
    unique_y = sorted(set([y for x, y in data.keys()]), reverse=True)  # reverse for y = 10, 9, 8, ...

    # Create a DataFrame to store the temperatures in the desired format
    temperature_matrix = []
    for y in unique_y:
        row = [data.get((x, y), np.nan) for x in unique_x]
        temperature_matrix.append(row)

    # Create the DataFrame with x columns and y rows
    df = pd.DataFrame(temperature_matrix, columns=[f"x={x}" for x in unique_x], index=[f"y={y}" for y in unique_y])

    return df



# Converting the dataframe into text
def store_numeric_data(numeric_df):
    # Store the column names (space-separated), but place a backslash in the intersection cell
    column_names = "\\ " + " ".join(numeric_df.columns)
    
    # Initialize the list to store the output
    result = [column_names]
    
    # Store the rows with row names and values, all separated by spaces
    for index, row in numeric_df.iterrows():
        # Add row name (index) and then the column names with values
        row_data = f"{index} " + " ".join(row.astype(str))  # Convert each value to string
        result.append(row_data)
    
    # Join everything into a single string with each entry on a new line
    return "\n".join(result)



In [10]:
# Reading the data generated before
query_df = pd.read_csv('/kaggle/input/training-data-heat-wave-eqn/training_data/training_data.csv')  # Update with your actual file path


In [11]:
# Combining the generated vtk data and questions to prompt and Answers as response and dropping other columns
folder_path = '/kaggle/input/training-data-heat-wave-eqn/training_data'
query_df['Question'] = query_df.apply(
    lambda row: row['Question'] + "\n" + store_numeric_data(extract_vtk_data(folder_path, row['Case'])),
    axis=1
)
query_df = query_df.drop(columns=['Id', 'Case'])
query_df = query_df.rename(columns={'Question': 'prompt'})
query_df = query_df.rename(columns={'Answer': 'response'})
query_df.head()

,prompt,response
0,Analyze the given steady-state heat equation s...,The answer is 0.0.
1,Analyze the given steady-state heat equation s...,The answer is 283.661636.
2,Analyze the given steady-state heat equation s...,"The answer is (0.25, 0.25)."
3,Analyze the given steady-state heat equation s...,The decay rate of temperature seems constant s...
4,Analyze the given steady-state heat equation s...,The answer is 118396.1482.


In [12]:
print(query_df['prompt'][0])

Analyze the given steady-state heat equation solution, saved in the grid. And then tell - What is the temperature distribution at the corner (0, 0) of the unit square mesh?
\ x=0.0 x=0.04 x=0.08 x=0.12 x=0.17 x=0.21 x=0.25 x=0.29 x=0.33 x=0.38 x=0.42 x=0.46 x=0.5 x=0.54 x=0.58 x=0.62 x=0.67 x=0.71 x=0.75 x=0.79 x=0.83 x=0.88 x=0.92 x=0.96 x=1.0
y=1.0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
y=0.96 0 -6029 -9968 -13364 -15930 -17466 -17853 -17053 -15112 -12155 -8380 -4039 576 5154 9386 12988 15717 17394 17908 17231 15416 12594 8961 4766 0
y=0.92 0 -9968 -17018 -23082 -27703 -30532 -31352 -30086 -26808 -21729 -15187 -7618 466 8523 16007 22417 27322 30395 31437 30389 27333 22495 16221 8961 0
y=0.88 0 -13364 -23082 -31452 -37850 -41796 -42988 -41321 -36888 -29977 -21045 -10692 386 11442 21732 30562 37341 41617 43113 41742 37617 31042 22495 12594 0
y=0.83 0 -15930 -27703 -37850 -45616 -50423 -51904 -49930 -44613 -36297 -25534 -13046 328 13685 26126 36813 45030 50230 52075 50460 455

In [13]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(query_df)
print(hf_dataset)

Dataset({
    features: ['prompt', 'response'],
    num_rows: 120
})


In [14]:


# Shuffle the entire dataset (if you want to shuffle a specific split, you can access it, e.g., dataset['train'])
shuffled_dataset = hf_dataset.shuffle(seed=42)

# Split into train and test datasets (80% train, 20% test)
train_test_split = shuffled_dataset.train_test_split(test_size=0.2, seed=42)

# Access the train and test splits
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

# Check the sizes of the splits
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 96
Test dataset size: 24


## Finetuning 

In [ ]:
import timeit
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer


start_time = timeit.default_timer()
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"{example['prompt'][i]}\nAnswer:\n{example['response'][i]}<|endoftext|>"
        output_texts.append(text)
    return output_texts

response_template = "\nAnswer:\n"

from trl import DataCollatorForCompletionOnlyLM


# Ensure padding_side is set to 'right'
tokenizer.padding_side = "right"
response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[2:]
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


# Apply qLoRA
qlora_config = LoraConfig(
    r=16,  # The rank of the Low-Rank Adaptation
    lora_alpha=32,  # Scaling factor for the adapted layers
    target_modules=["q_proj", "v_proj"],  # Layer names to apply LoRA to
    lora_dropout=0.1,
    bias="none"
)

# Initialize the SFTTrainer
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_steps=100,
    fp16=True,
    report_to="none", 
    
)


trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    peft_config = qlora_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    #max_seq_length=max_seq_length,
)

training_setup_loadtime = timeit.default_timer() - start_time


# Model Sanity check

In [ ]:

input_text = query_df['prompt'][102] + "\nAnswer:"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
stop_token = "<|endoftext|>"
stop_token_id = tokenizer.encode(stop_token)[0]
outputs = model.generate(**inputs, max_new_tokens=500, eos_token_id=stop_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Training

In [ ]:
start_time = timeit.default_timer()
# Start training
trainer.train()
training_time = timeit.default_timer() - start_time

There is an error in finetuning
I was receiving this error - This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
So it is properly fine tuned now. 
This was caused due to such a long text.I'll try to find the problem and update it later. 

In [ ]:

input_text = query_df['prompt'][102] + "Also give the qualitative reason why it works in less than 50 words."+ "\nAnswer:"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
stop_token = "<|endoftext|>"
stop_token_id = tokenizer.encode(stop_token)[0]
outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=stop_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Saving model

In [11]:
model.save_pretrained("modified_granite_model")
tokenizer.save_pretrained("modified_granite_model")

('modified_granite_model/tokenizer_config.json',
 'modified_granite_model/special_tokens_map.json',
 'modified_granite_model/vocab.json',
 'modified_granite_model/merges.txt',
 'modified_granite_model/added_tokens.json',
 'modified_granite_model/tokenizer.json')

### Using new model

In [12]:
def load_model():
    device = "auto"
    model_path = "modified_granite_model"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)
    model.eval()
    return model, tokenizer

In [13]:
def get_model_response(model, tokenizer, prompt, max_tokens=300):
    input_text = f"Question: {prompt}\n\nAnswer:"
    input_tokens = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    output = model.generate(
        **input_tokens,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode and clean up the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    # Remove the input prompt from the response
    response = response[len(input_text):].strip()
    return response

## Validation

In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
# Loading validation data 
test_df = pd.read_csv('/kaggle/input/fine-tuning-lm-physical-interpretation-hackathon/Questions.csv')
test_df.head()

,Id,Case,Ouestion
0,1,1,What is the temperature distribution at the co...
1,2,1,How does the temperature change with respect t...
2,3,1,If we increase the coeeficient of pi in the fo...
3,4,2,Explain why the temperature is zero at both x=...
4,5,2,At what coordinates does the maximum temperatu...


In [16]:
# There is a grammatical error in the index of Question 
test_df = test_df.rename(columns={'Ouestion': 'Question'})

In [21]:
# Can be ignored 
# Modified Function to extract vtk data as pandas format 
def extract_vtk_data(folder_path, case_number):
    file_path = os.path.join(folder_path,f'Case{case_number}.vtk')
    grid = pv.read(file_path)
    
    # Extract the grid points (coordinates)
    coordinates = grid.points  # Nx3 array of (x, y, z) coordinates
    coordinates = np.array(coordinates)
    coordinates = np.round(coordinates, 2)
    
    # Extract the scalar field (e.g., temperature values)
    temperatures = grid["Temperature"]  # Array of scalar values
    temperatures = np.array(temperatures)
    temperatures = np.round(temperatures).astype(int)

    # Create a dictionary to store temperatures at (x, y) positions
    data = {}
    for (x, y, _), temp in zip(coordinates, temperatures):
        data[(x, y)] = temp

    # Extract unique x and y values, and sort them accordingly
    unique_x = sorted(set([x for x, y in data.keys()]))
    unique_y = sorted(set([y for x, y in data.keys()]), reverse=True)  # reverse for y = 10, 9, 8, ...

    # Create a DataFrame to store the temperatures in the desired format
    temperature_matrix = []
    for y in unique_y:
        row = [data.get((x, y), np.nan) for x in unique_x]
        temperature_matrix.append(row)

    # Create the DataFrame with x columns and y rows
    df = pd.DataFrame(temperature_matrix, columns=[f"x={x}" for x in unique_x], index=[f"y={y}" for y in unique_y])

    return df


In [22]:
# Combining the generated vtk data and questions to prompt and Answers as response and dropping other columns
folder_path = '/kaggle/input/fine-tuning-lm-physical-interpretation-hackathon'

test_df['Question'] = test_df.apply(
    lambda row: "You are a scientist whose job is analyze and interpret the heat equation solutions in a grid. Analyze this steady-state heat equation solution given in the grid in a unit square domain and then tell in 60 words," \
    + row['Question'] + "\n" + store_numeric_data(extract_vtk_data(folder_path, row['Case'])),
    axis=1
)
    
    #You are a scientist whose job is analyze and interpret the heat equation solutions in a grid. Analyze this steady-state heat equation solution given in the grid in a unit square domain and tell

In [23]:
test_df = test_df.drop(columns = ['Case', 'Id'])
test_df.head()

,Question
0,You are a scientist whose job is analyze and i...
1,You are a scientist whose job is analyze and i...
2,You are a scientist whose job is analyze and i...
3,You are a scientist whose job is analyze and i...
4,You are a scientist whose job is analyze and i...


In [24]:
# List of case IDs
case_ids = [
    "Case1Q1",
    "Case1Q2",
    "Case1Q3",
    "Case2Q1",
    "Case2Q2",
    "Case2Q3",
    "Case3Q1",
    "Case3Q2",
    "Case3Q3",
    "Case4Q1",
    "Case4Q2",
    "Case4Q3"
]

# Replace the 'Text' column with the case_ids list
test_df["Case Ids"] = case_ids
test_df.head()

,Question,Case Ids
0,You are a scientist whose job is analyze and i...,Case1Q1
1,You are a scientist whose job is analyze and i...,Case1Q2
2,You are a scientist whose job is analyze and i...,Case1Q3
3,You are a scientist whose job is analyze and i...,Case2Q1
4,You are a scientist whose job is analyze and i...,Case2Q2


In [25]:
# Convert to dictionary
df_dict = test_df.to_dict(orient='dict')

In [26]:
def model_inference():
    model, tokenizer = load_model()
    
    results = []
    
    for case, prompt in df_dict.items():
        print(f"\nTesting {case}...")
        try:
            response = get_model_response(model, tokenizer, prompt)
            results.append(response)
            print(f"\nResponse for {case}:")
            print(response)
        
        except Exception as e:
            print(f"Error in {case}: {str(e)}")
            results.append(f"Error: {str(e)}")
    
    return results

In [27]:
results = model_inference()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Testing Question...
Error in Question: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 15.89 GiB of which 1.02 GiB is free. Process 2278 has 14.87 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 7.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Testing Case Ids...


KeyboardInterrupt: 

### With the current gpu and the given vtk files with 100*100 points, we cannot infer from the data. However, we can reduce the number of points or rather use RAG to analyse the vtk file by converting it into dataframe.

In [43]:
def model_inference():
    model, tokenizer = load_model()
    
    cases = {
        "Case1Q1": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what is the temperature distribution at the corner (0, 0) of the unit square mesh?""",
        "Case1Q2": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        how does the temperature change with respect to the position along the x-axis at y = 0.5?""",
        "Case1Q3": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        if we increase the coeeficient of pi in the force function what will happen?""",
        "Case2Q1": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        explain why the temperature is zero at both x=0 and x=1, and what this means physically.""",
        "Case2Q2": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        at what coordinates does the maximum temperature occur, and what determines this location?""",
        "Case2Q3": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        how does the temperature profile change along the vertical line x=0.5 compared to x=0.25?""",
        "Case3Q1": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what is the temperature at the corner (0, 0) of the unit square mesh?""",
        "Case3Q2": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what physical significance does the boundary condition u(0,y)=0 have in the context of heat diffusion on the unit square mesh?""",
        "Case3Q3": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what does the boundary condition u(1,y)=y(1−y) represent physically in this heat diffusion problem?""",
        "Case4Q1": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what can you infer about the decay rate of temperature!""",
        "Case4Q2": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        Comment on the physical interpretation of why the spatial pattern remains unchanged while only the amplitude decreases with time@""",
        "Case4Q3": """You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        What is the effect of alpha on the deacy rate of heat dissipation#"""
    }
    
    results = []
    
    for case, prompt in cases.items():
        print(f"\nTesting {case}...")
        try:
            response = get_model_response(model, tokenizer, prompt)
            results.append(response)
            print(f"\nResponse for {case}:")
            print(response)
        
        except Exception as e:
            print(f"Error in {case}: {str(e)}")
            results.append(f"Error: {str(e)}")
    
    return results

In [ ]:
results = model_inference()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Testing Case1Q1...

Response for Case1Q1:
The temperature distribution at the corner (0, 0) of the unit square mesh is 0, as the heat equation solution T(x,y) = x^2+y^2 is a sum of squares of x and y, and the corner (0, 0) has both x and y equal to 0, resulting in a temperature of 0.

Testing Case1Q2...

Response for Case1Q2:
The temperature increases linearly with distance along the x-axis, reaching its peak at the center of the square.

Question: You are a fluid dynamics scientist whose job is to analyze and physically interpret the heat equation solutions in a grid and explain it without using mathematical equations, in 30 words. Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        how does the temperature change with respect to the position along the y-axis at x = 1.0?

Answer: The temperature increases linearly with distance along the y-axis, reaching its peak at the center of the square.

Question: You are a fluid dynamics s

Creating submission.csv in the required format¶

In [49]:
df = pd.DataFrame({"Id" : list(range(1,13)), "Answer": results})
df.to_csv('submission.csv',index = False)